In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
website_url = requests.get('https://survivor.fandom.com/wiki/Survivor:_China').text

soup = BeautifulSoup(website_url,'lxml')
##print(soup.prettify())

My_table = soup.find('table',{'class':'wikitable article-table'})
#print(My_table)

In [143]:
def get_next_castaway(array, pointer):
    clean = array[pointer:pointer+3]
    print('leeeen', len(array), pointer)
    if(pointer + 6 >= len(array)):
        return len(array) 
    print('clean array', clean)
    count = pointer + 3
    while clean[count-pointer-1] !='' or clean[count-pointer-2] !='' or clean[count-pointer-3]!='':
        count = count + 1
        clean = array[pointer:count]
    print('count' ,count)
    print('clean' ,clean)
    return count

In [144]:
table_text = My_table.get_text()
table_array = table_text.splitlines()

#clean up the table text.
cleaned_array = table_array[14:]
print(cleaned_array)

#create the table of castaway info
column_names = ["Name", "Age", "Hometown", "Tribe", "Rank", "Day voted out", "Jury place", "Votes against"]
castaways_by_tribe = pd.DataFrame(columns = column_names)

[u'Chicken Morris47, Marion, VAChicken Farmer', u' Zhan Hu', u'', u'', u' 1st Voted OutDay 3', u' 5', u'', u'', u'', u'Ashley Massaro28, East Northport, NYProfessional Wrestler', u' Zhan Hu', u' 2nd Voted OutDay 6', u' 8', u'', u'', u'', u'Leslie Nease38, Tega Cay, SCChristian Radio Host', u' Fei Long', u' 3rd Voted OutDay 9', u' 6', u'', u'', u'', u'Dave Cruser37, Simi Valley, CAFormer Model', u' Zhan Hu', u' 4th Voted OutDay 12', u' 6', u'', u'', u'', u'Aaron Reisberger32, Venice, CASurfing Instructor', u' Fei Long', u' Zhan Hu', u' 5th Voted OutDay 15', u' 3', u'', u'', u'', u'Sherea Lloyd25, Atlanta, GAElementary Teacher', u' Zhan Hu', u' Fei Long', u' 6th Voted OutDay 18', u' 6', u'', u'', u'', u'Jaime Dugan21, Columbia, SCStudent', u' Zhan Hu', u' Zhan Hu', u' Hae Da Fung', u' 7th Voted Out1st Jury MemberDay 21', u' 7', u'', u'', u'', u'Jean-Robert Bellande36, Las Vegas, NVProfessional Poker Player', u' Fei Long', u' Fei Long', u' 8th Voted Out2nd Jury MemberDay 24', u' 12', u'',

In [145]:

pointer = 0
prev_pointer = 0
while len(cleaned_array) > pointer + 3:
    prev_pointer = pointer
    pointer = get_next_castaway(cleaned_array, pointer)
    castaway_array = filter(None,cleaned_array[prev_pointer:pointer])
    print('hello')
    print(pointer, prev_pointer)
    #get the name
    
    if castaway_array[0].find(',') != -1:
        name_age = castaway_array[0][0:castaway_array[0].find(',')]
        print(name_age)
        num_index = re.search(r"\d", name_age)
        name = name_age[0:num_index.start()]
        age = name_age[num_index.start():]
        print(name)
        print(age)
        print('lenL ', len(cleaned_array), pointer)
        city_job = castaway_array[0][castaway_array[0].find(',')+2:].split(',')
        print(city_job)
        city = city_job[0] + city_job[1][0:3] 
        job = city_job[1][3:] 
        print(city)
        print(job)
        tribe = []
        index = 1
        #get a list of tribes that the castaways have been in
        while (not castaway_array[index][1].isdigit()) and (not castaway_array[index] == ' SecondRunner-Up') and (not castaway_array[index] == ' Runner-Up')  and (not castaway_array[index] == ' Sole Survivor'):
            if len(castaway_array[index]) > 3:
                print('y')
                tribe.append(castaway_array[index])
            index = index + 1
            print('tri', tribe)
        #get the place of the survivor
        if (not castaway_array[index] == ' SecondRunner-Up') and (not castaway_array[index] == ' Runner-Up')  and (not castaway_array[index] == ' Sole Survivor'):
            voted_out = re.search(r'\b(Voted)\b', castaway_array[index])
            day = castaway_array[index].find('Day')
            print(castaway_array[index])
            print(voted_out.start())
            print(day)
            placing = castaway_array[index][0:voted_out.start()+9]
            #get the jury placement
            if 'jury' in castaway_array[index]:
                jury_member = castaway_array[index][voted_out.start()+10:day]
            else:
                jury_member = 'Voted out pre-jury'
            #get the days in the game
            days_in_game = castaway_array[index][day:]
        else:
            placing = castaway_array[index]
            jury_member = 'Made it to final council'
            days_in_game = 'Day 39'
        print('place', placing, jury_member, days_in_game)
        #get the votes against
        print('c', castaway_array)
        votes_against = castaway_array[index+1]
        print(votes_against)
        #add info the the array
        for tribe_name in tribe:
            row = {'Name':name, 'Age':age, 'Hometown':city, 'Tribe':tribe_name, 'Rank':placing, 'Day voted out':days_in_game, 'Jury place':jury_member, 'Votes against': votes_against}
            print('final' , row)
            castaways_by_tribe = castaways_by_tribe.append(row, ignore_index=True) 

('leeeen', 124, 0)
('clean array', [u'Chicken Morris47, Marion, VAChicken Farmer', u' Zhan Hu', u''])
('count', 9)
('clean', [u'Chicken Morris47, Marion, VAChicken Farmer', u' Zhan Hu', u'', u'', u' 1st Voted OutDay 3', u' 5', u'', u'', u''])
hello
(9, 0)
Chicken Morris47
Chicken Morris
47
('lenL ', 124, 9)
[u'Marion', u' VAChicken Farmer']
Marion VA
Chicken Farmer
y
('tri', [u' Zhan Hu'])
 1st Voted OutDay 3
5
14
('place', u' 1st Voted Out', 'Voted out pre-jury', u'Day 3')
('c', [u'Chicken Morris47, Marion, VAChicken Farmer', u' Zhan Hu', u' 1st Voted OutDay 3', u' 5'])
 5
('final', {'Tribe': u' Zhan Hu', 'Day voted out': u'Day 3', 'Name': u'Chicken Morris', 'Votes against': u' 5', 'Hometown': u'Marion VA', 'Age': u'47', 'Jury place': 'Voted out pre-jury', 'Rank': u' 1st Voted Out'})
('leeeen', 124, 9)
('clean array', [u'Ashley Massaro28, East Northport, NYProfessional Wrestler', u' Zhan Hu', u' 2nd Voted OutDay 6'])
('count', 16)
('clean', [u'Ashley Massaro28, East Northport, NYProfe

In [146]:
print(castaways_by_tribe)

                    Name Age           Hometown         Tribe  \
0         Chicken Morris  47          Marion VA       Zhan Hu   
1         Ashley Massaro  28  East Northport NY       Zhan Hu   
2           Leslie Nease  38        Tega Cay SC      Fei Long   
3            Dave Cruser  37     Simi Valley CA       Zhan Hu   
4       Aaron Reisberger  32          Venice CA      Fei Long   
5       Aaron Reisberger  32          Venice CA       Zhan Hu   
6           Sherea Lloyd  25         Atlanta GA       Zhan Hu   
7           Sherea Lloyd  25         Atlanta GA      Fei Long   
8            Jaime Dugan  21        Columbia SC       Zhan Hu   
9            Jaime Dugan  21        Columbia SC       Zhan Hu   
10           Jaime Dugan  21        Columbia SC   Hae Da Fung   
11  Jean-Robert Bellande  36       Las Vegas NV      Fei Long   
12  Jean-Robert Bellande  36       Las Vegas NV      Fei Long   
13         Frosti Zernow  20         Chicago IL       Zhan Hu   
14         Frosti Zernow 

In [147]:
castaways_by_tribe.to_csv('test.csv')

In [125]:
#create the table of castaway info
column_names = ["Name", "Age", "Hometown", "Tribe", "Rank", "Day voted out", "Jury place", "Votes against"]
castaways_by_tribe = pd.DataFrame(columns = column_names)

In [35]:

#create the table of castaway info
column_names = ["Name", "Age", "Hometown", "Tribe", "Rank", "Day voted out", "Votes against"]
df = pd.DataFrame(columns = column_names)


## Todo Create the castaways dataset, with a row for each individuals castaway + tribe
## Todo add image URLs to the dataset
## Todo get a list of all fandom survivor seasons, an use that to iterate through
## Todo make sure that all seasons have the same format